In [1]:
#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [2]:

# Dataset is already loaded below
data = pd.read_csv("employee.csv")

In [3]:
# Dimensions of training data
data.shape

(4277, 13)

In [4]:
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [5]:

# drop id, timestamp and country columns
data = data.drop(columns=['id', 'timestamp','country'])

In [6]:

# Explore columns
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [7]:
# replace NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [8]:

#Handling null values in categorical columns
data = data.dropna()

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


In [10]:
# create another copy of dataset and append encoded features to it
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [11]:
# select categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols


['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [12]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [13]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape


(4261, 25)

In [14]:

final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [15]:

final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


Train Test Split

In [16]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


In [17]:

# select numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols


['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [18]:
# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [19]:

X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
681,-0.751071,1,2.335756,-0.632382,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
705,0.100456,1,-0.448658,2.405617,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
3582,-0.042766,0,0.182754,2.405617,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
976,-1.254934,0,-0.497743,-0.632382,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1061,-0.247209,0,-1.631143,-0.632382,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,-0.133654,0,-0.254896,-0.632382,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1008,-0.209635,0,0.685796,-0.632382,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
1889,-1.758797,0,-0.497743,-0.632382,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3230,-0.226682,0,0.014808,-0.632382,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [20]:

#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)


LinearRegression()

In [21]:
reg.coef_

array([  201.50185153,    70.52755309,   183.86329193,   228.45715954,
          49.29722152,   -56.38404901,   155.245367  , -2102.00333161,
        1080.16146819,   932.380799  ,   579.20743299,   612.62527108,
        1081.25626484,   895.37471924,   145.07997577,  1032.0782794 ,
         752.62748105,  3655.73443185,   384.18917881,  -238.83421202,
         221.16727203,   238.67637829,  -286.96593868,  -189.9374902 ])

In [22]:

reg.intercept_


6074.645371626479

In [23]:
# Normalized MSE (Dividing by mean)
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

205.49531885917494

In [24]:
# Predict on the test data
y_pred = reg.predict(X_test)

In [25]:
#Evaluate the model on test data
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)

Mean Squared Error: 10729.27906227546


In [26]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [27]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
4238,0.224463,1,0.074803,2.402373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1887,-0.712771,1,-2.206926,-0.602354,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1941,-0.363669,1,0.427344,-0.602354,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2952,0.009470,0,0.073196,-0.602354,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
3097,-0.276112,1,-0.804411,-0.001409,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,-0.216880,0,-0.450323,2.402373,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3870,-0.019032,0,-0.450323,-0.602354,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2052,-1.704553,0,2.477347,0.599536,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3790,0.279011,1,0.720745,-0.602354,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0


In [28]:
y_hat = reg.predict(X_test)

In [29]:
#Predicted Values
y_hat

array([6466.02944827, 6265.0403601 , 7188.78347526, 6759.2764943 ,
       7307.17964089, 7111.22874349, 7442.4611212 , 6517.48750713,
       5731.93186981, 7677.64670226, 7671.15998548, 7078.12725464,
       7349.54605605, 7186.30918105, 7612.1721121 , 7319.75362921,
       6194.42448807, 6584.33443752, 6643.65495475, 7798.02595374,
       6611.14348459, 5971.95816144, 7380.67299495, 6943.61379612,
       7182.29017442, 7857.63576295, 7205.54549493, 7161.08735807,
       7537.00417042, 6508.9403351 , 6431.93595024, 7282.31279041,
       7143.61388171, 6684.51499687, 7007.62051912, 7037.0889847 ,
       6923.36429945, 6361.78181085, 7514.84593879, 7618.0183783 ,
       6884.20235946, 7037.0889847 , 6796.22748563, 7472.81863252,
       7351.82403201, 6481.06633762, 7290.18502702, 6554.31155001,
       8061.98557361, 7443.29369604, 6864.70187509, 7212.51229512,
       6972.86004708, 7252.31393992, 7078.12725464, 6770.84153354,
       6686.6943069 , 7078.12725464, 7028.07000563, 7813.87365

In [30]:
# Normalized Mean Absolute Error (Dividing by mean)
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ",mae_test)

Test Mean Abosulte Error:  0.1251141330937557


In [31]:
# Normalized Mean Squared Error (Dividing by mean)
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse_test)

Test Mean Squared Error:  224.03582024426098
